# Prepare data for training

1. Load all embeddings
1. Tokenize tweets
1. Construct embdding matrix matchin the tokenizer
1. Generate sequences
1. Write everything to disk

In [1]:
from keras.preprocessing.text import Tokenizer
import pandas as pd
import numpy as np
import sys

sys.path.append("../python")

import utils


/home/musella/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
max_words = 2500
glove_dim = 25
hash_dim = 5

In [3]:
words_embed = utils.load_glove_embedding(utils.glove_embedding_path(25))
hash_embed = utils.load_glove_embedding('../data/models/hashtags/hash_vectors.d'+str(hash_dim)+'.txt')

In [4]:
df = pd.read_hdf("../data/preproc/tweets.hd5")

tk = Tokenizer(num_words=max_words,
               filters="", # already applied
               lower=True,
               split=" ",oov_token="<unk>")
tk.fit_on_texts(df["preproc_text"]) 

In [5]:
reload(utils)
words_embed_mtx, words_unk = utils.fill_embedding_matrix(tk.word_index,words_embed,max_words,glove_dim)
hash_embed_mtx, hash_unk = utils.fill_embedding_matrix(tk.word_index,hash_embed,max_words,hash_dim)


In [6]:
unk = { word: tk.word_index[word] for word in set(words_unk).intersection(set(hash_unk)) }
print(unk)

stop_mtx = np.zeros( (max_words+2, 2) )

# stop_mtx[unk.pop("<trunc>"),-2] = 1.
stop_mtx[unk.pop("<stop>"),-1] = 1.

stop_mtx[list(unk.values()),-2] = 1.


{'h2o': 1862, 'datasciencectrl': 1469, 'kirkdborne': 500, 'poptimize': 809, 'chatbot': 1064, 'ipfconline1': 550, 'interpretability': 1302, 'machinelearning': 484, 'classifier': 2366, 'vanloon': 151, 'neurips': 563, 'datascience': 2044, 'chatbots': 941, 'variational': 2489, 'nodexl': 1582, 'probabilistic': 1747, 'piccard': 2495, 'deeplearn007': 818, 'evankirstel': 808, 'spirosmargaris': 784, 'daysofmlcode': 864, 'fisher85m': 900, 'jblefevre60': 1195, 'bengio': 1871, 'bodyproblem': 1749, 'kaggle': 2058, 'daysofcode': 527, 'antgrasso': 1135, 'freshsalis': 1337, 'whova': 664, 'cloudexpo': 2386, 'alphazero': 2420, 'jimmarous': 1258, 'automl': 1844, 'mikequindazzi': 165, 'neurips2018': 2312, 'pulipaka': 1554, 'v1': 925, 'convolutional': 859, 'pierrepinna': 2295, 'kdnuggets': 931, 'sagemaker': 2346, '<stop>': 5, 'ahier': 1381, 'analyticbridge': 2394, 'tensorflow': 348, 'pytorch': 1658, 'intengineering': 2352, 'scikit': 1692, 'datax': 2458}


In [7]:
unk_idx = list(unk.values())
unk

{'ahier': 1381,
 'alphazero': 2420,
 'analyticbridge': 2394,
 'antgrasso': 1135,
 'automl': 1844,
 'bengio': 1871,
 'bodyproblem': 1749,
 'chatbot': 1064,
 'chatbots': 941,
 'classifier': 2366,
 'cloudexpo': 2386,
 'convolutional': 859,
 'datascience': 2044,
 'datasciencectrl': 1469,
 'datax': 2458,
 'daysofcode': 527,
 'daysofmlcode': 864,
 'deeplearn007': 818,
 'evankirstel': 808,
 'fisher85m': 900,
 'freshsalis': 1337,
 'h2o': 1862,
 'intengineering': 2352,
 'interpretability': 1302,
 'ipfconline1': 550,
 'jblefevre60': 1195,
 'jimmarous': 1258,
 'kaggle': 2058,
 'kdnuggets': 931,
 'kirkdborne': 500,
 'machinelearning': 484,
 'mikequindazzi': 165,
 'neurips': 563,
 'neurips2018': 2312,
 'nodexl': 1582,
 'piccard': 2495,
 'pierrepinna': 2295,
 'poptimize': 809,
 'probabilistic': 1747,
 'pulipaka': 1554,
 'pytorch': 1658,
 'sagemaker': 2346,
 'scikit': 1692,
 'spirosmargaris': 784,
 'tensorflow': 348,
 'v1': 925,
 'vanloon': 151,
 'variational': 2489,
 'whova': 664}

In [8]:
np.isnan(words_embed_mtx).sum()

0

In [9]:
np.isnan(hash_embed_mtx).sum()

0

In [11]:
! mkdir ../data/models/sequences

import pickle
import json


with open("../data/models/sequences/info.json","w+") as out:
    save = dict(max_words=max_words,unk_idx=unk_idx)
    out.write(json.dumps(save))
    
with open('../data/models/sequences/tokenizer.pkl','wb+') as out:
    out.write( pickle.dumps(tk) )
    
np.save('../data/models/sequences/embed_mtx.npy',words_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',hash_embed_mtx)
np.save('../data/models/sequences/hash_mtx.npy',stop_mtx)

    


In [12]:
from sklearn.model_selection import train_test_split

df["sequences"] = tk.texts_to_sequences(df["preproc_text"])

df.to_hdf("../data/preproc/sequences.hd5","squences",columns=["sequences"],mode="w")

# index = np.arange(df.index.shape[0]).astype(np.int)
# train_idx, test_idx = train_test_split(index,test_size=0.2,random_state=123456)
# df_train = df.iloc[train_idx]
# df_test = df.iloc[test_idx]
# df_train.to_hdf("../data/preproc/sequences.hd5","train",columns=["sequences"])
# df_test.to_hdf("../data/preproc/sequences.hd5","test",columns=["sequences"])


/home/musella/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:1471: PerformanceWarning: 
your performance may suffer as PyTables will pickle object types that it cannot
map directly to c-types [inferred_type->mixed,key->block2_values] [items->['extended_tweet/full_text', 'lang', 'retweeted_status/extended_tweet/full_text', 'text', 'user/name', 'preproc_text', 'sequences']]

  return pytables.to_hdf(path_or_buf, key, self, **kwargs)


In [13]:
list( zip(df.iloc[:20]["text"],tk.sequences_to_texts( tk.texts_to_sequences(df.iloc[:20]["preproc_text"]) )) )

[('#neurips2018 attendees, a new discussion on "HCI and monitization in A.I." was just started. Join the conversation on the Whova app! https://t.co/VZVHTRVN4U',
  '<hashtag> #neurips2018 attendees a new discussion on <unk> and <unk> in a i was just started join the conversation on the whova app <url> <stop>'),
 ('Interesting stats about my #NeurIPS2018 tweets! People mostly tend to check the NeurIPS tweets exactly after the conf. (8-10PM, 1st col.) or just before the day starts (6-8Am, 6th col.)  \U0001f928 https://t.co/a7aFg4g7na',
  'interesting stats about my <hashtag> #neurips2018 tweets people mostly <unk> to check the neurips tweets exactly after the <unk> <number> <number> pm <number> st <unk> or just before the day starts <number> <number> am <number> th <unk> <url> <stop>'),
 ('Jordan Gruber @j2g2_ : Our robot "Marvin" can count and identify asbestos particles - this can cut costs to $15 - versus $50 - per analysis. @FrontierMicro  #DeepLearning #innovation #artificialintelli

In [14]:
# ?tk.texts_to_sequences

In [15]:
tk.texts_to_sequences( ["<unk>"] )

[[1]]

In [ ]:
# tk.word_index